In [1]:
%reload_ext sql
%reload_ext lib.sqlite.sqlite_evaluate_magic

ModuleNotFoundError: No module named 'lib'

In [2]:
import os

## Connecting to Menu Databasse

In [3]:
publications_db_url = 'sqlite:///' + os.path.expanduser('menus.db')
%sql $publications_db_url

'Connected: @menus.db'

## Checking Table

In [4]:
%%sql 
select * from menu limit 1;

 * sqlite:///menus.db
Done.


id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count
12463,,HOTEL EASTMAN,BREAKFAST,COMMERCIAL,"HOT SPRINGS, AR",CARD; 4.75X7.5;,EASTER,,1900-2822,,,1900-04-15,Hotel Eastman,,,,complete,2,67


In [5]:
%%sql 
select * from dish limit 1;

 * sqlite:///menus.db
Done.


id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
1,"Consomme, Printaniere Royal",,8,8,1897,1927,0.2,0.4


## Check for uniqueness of id fields

### Menu Table

In [6]:
%%sql

select *
from menu
where id in (select id as c 
from menu 
group by id
having count(id) > 1 )

 * sqlite:///menus.db
Done.


id,name,sponsor,event,venue,place,physical_description,occasion,notes,call_number,keywords,language,date,location,location_type,currency,currency_symbol,status,page_count,dish_count


### Dish Table

In [7]:
%%sql

select *
from dish
where id in (select id as c 
from dish 
group by id
having count(id) > 1 )

 * sqlite:///menus.db
Done.


id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


### Menupage Table

In [8]:
%%sql

select *
from menupage
where id in (select id as c 
from menupage 
group by id
having count(id) > 1 )

 * sqlite:///menus.db
Done.


id,menu_id,page_number,image_id,full_height,full_width,uuid


### Menu Item Table

In [17]:
%%sql

select *
from menuitem
where id in (select id as c 
from menuitem 
group by id
having count(id) > 1 )

 * sqlite:///menus.db
Done.


id,menu_page_id,price,high_price,dish_id,created_at,updated_at,xpos,ypos


### Each Menu item should have a single menu page id

In [20]:
%%sql


select id, menu_page_id
from menuitem
where menu_page_id in (select id as c 
from menuitem 
group by id
having count(menu_page_id) > 1 )

 * sqlite:///menus.db
Done.


id,menu_page_id


### Dish Count less than me
nu page count

Ideally, dish count should be more than menu page count. But there are some instancs where this is not correct. This could be due to other details and pictures in the menu

In [38]:
%%sql

select count(*)
from menu
where cast(page_count as integer) > cast(dish_count as integer)

 * sqlite:///menus.db
Done.


count(*)
150


### Lowest Price Should not be greater than highest price

In [41]:
%%sql

select *
from dish
where cast(lowest_price as integer) > cast(highest_price as integer)

 * sqlite:///menus.db
Done.


id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price


### First appeared shold not be greater than last appeared

In [42]:
%%sql

select *
from dish
where cast(first_appeared as integer) > cast(last_appeared as integer)

 * sqlite:///menus.db
Done.


id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
164029,Clear beef broth,,0,1,1900,0,0.25,0.25
204888,Hot Roast Beef with Gravy,,0,1,1900,0,0.25,0.25
250693,"SURI LEBERLI - Shredded Calf's Liver Flambe in White Wine, Roesti",,0,1,1945,0,,
250699,"SWISS MINCED VEAL, ROESTI",,0,1,1945,0,,
301736,Cafe Glacee,,0,2,1940,0,0.4,0.4
309629,Garlic Butter,,0,1,1947,0,0.4,0.4


### Top 5 dishes with most appearances 

In [9]:
%%sql

select name, count(name) 
from dish 
group by name 
order by count(name) desc
limit 5

 * sqlite:///menus.db
Done.


name,count(name)
Fried sweet potatoes,49
EGGS poached on toast (2),48
Cold Roast Beef,47
"Grape Fruit, Half",45
G H Mumm's Extra Dry,45


### Top 5 dishes appearing on most number of menus

In [10]:
%%sql

select id, name, menus_appeared 
from dish 
order by menus_appeared desc
limit 5

 * sqlite:///menus.db
Done.


id,name,menus_appeared
150,Little Neck clams,995
1034,Anchovies on toast,99
1047,*Roast Capon,99
1372,Creme Yvette,99
2798,Pork Tenderloin,99


# Let's see when a dish was appeared on which menu, sponsor, event, place (limiting to 10 counts)

In [12]:
%%sql

select dish.name, menu.sponsor, menu.event, menu.place, dish.first_appeared, menu.date
from menu inner join menupage on menu.id = menupage.menu_id
inner join menuitem on menupage.id = menuitem.menu_page_id
inner join dish on menuitem.dish_id = dish.id
where dish.name = 'Fried Eggs'
limit 10


 * sqlite:///menus.db
Done.


name,sponsor,event,place,first_appeared,date
Fried Eggs,CUNARD LINE,BREAKFAST,SS ETRURIA,1856,1900-02-21
Fried Eggs,,,,1856,1900-07-08
Fried Eggs,,,,1856,1900-07-08
Fried Eggs,,,,1856,1900-07-01
Fried Eggs,BATTERY PARK HOTEL,BREAKFAST,"ASHEVILLE,NC",1856,1900-02-21
Fried Eggs,,,,1856,1900-06-01
Fried Eggs,,,,1856,1900-07-08
Fried Eggs,CANADIAN PACIFIC RAILWAY COMPANY,BREAKFAST,RMS EMPRESS OF CHINA,1856,1900-04-23
Fried Eggs,,,,1856,1901-11-07
Fried Eggs,,,,1856,1901-10-05


# Let's see when a dish was first appered on which menu, sponser, event, place

In [13]:
%%sql

select dish.name, menu.sponsor, menu.event, menu.place, menu.date, dish.first_appeared
from menu inner join menupage on menu.id = menupage.menu_id
inner join menuitem on menupage.id = menuitem.menu_page_id
inner join dish on menuitem.dish_id = dish.id
where dish.name = 'French Fries'
and menu.date is not ''
order by menu.date
limit 1

 * sqlite:///menus.db
Done.


name,sponsor,event,place,date,first_appeared
French Fries,Hotel Worthy,DINNER,"SPRINGFIELD, MASS.",1901-07-05,1901


# Let's see when a dish was last appered on which menu, sponser, event, place

In [14]:
%%sql

select dish.name, menu.sponsor, menu.event, menu.place, menu.date, dish.last_appeared
from menu inner join menupage on menu.id = menupage.menu_id
inner join menuitem on menupage.id = menuitem.menu_page_id
inner join dish on menuitem.dish_id = dish.id
where dish.name = 'Fried Eggs'
and menu.date is not ''
order by menu.date desc
limit 1

 * sqlite:///menus.db
Done.


name,sponsor,event,place,date,last_appeared
Fried Eggs,Holland America Cruises,,,1987-01-01,1987


### Each dish should appear on a Menu

In [135]:
%%sql


select count(distinct dish.id)
from menu inner join menupage on menu.id = menupage.menu_id
inner join menuitem on menupage.id = menuitem.menu_page_id
inner join dish on menuitem.dish_id = dish.id

 * sqlite:///menus.db
Done.


count(distinct dish.id)
413380


In [138]:
%%sql

select count(dish.id)
from dish

 * sqlite:///menus.db
Done.


count(dish.id)
428132
